In diesem Notebook möchten wir unseren Datensatz für das trainieren der Modelle vorbereiten. Dazu müssen wir zum Beispiel einige Variablen Dummy kodieren.

In [ ]:
import pandas as pd

df = pd.read_csv('/content/DF_Join_final_bereinigt (3).csv')
df


In [ ]:
import numpy as np

In [ ]:
#Zunächst entfernen wir noch die Spalte imdb_id, die wir nicht ins Modell einfließen lassen

df = df.drop(['imdb_id'], 1)

In [ ]:
#Zunächste formatieren wir die Namen der Schauspieler um, um später die Top-100 Schauspielernamen richtig formatiert als Spaltennamen zu haben
df['name'] = df['name'].str.replace(' ', '_')
df


#Gruppieren auf Filmtitel

Madeleine

In [ ]:
# Aggregatfunktionen für bestimmte Spalten angeben
agg_functions = {
    'name': lambda x: ', '.join(x),
    'character': lambda x: ', '.join(x),
    'role': 'first',  #hier die Kontrolle nach oben ziehen
    'description': 'first',
    'runtime': 'mean',
    'genres': 'first',
    'production_countries': 'first',
    'imdb_score': 'mean',
    'imdb_votes': 'mean',
    'title': 'first',
    'tmdb_score': 'mean',



}
# Gruppierung nach der Spalte 'title' und Anwendung der angegebenen Aggregatfunktionen
df = df.groupby('title', as_index=False).agg(agg_functions)
df

In [ ]:
df.isna().sum()

#Dummykodierung:

###Zunächst Dummy kodieren wir die Spalte genres

Leonie

In [ ]:
#In der Spalte Genres wollen wir jeweils nur den ersten Eintrag behalten

df['genres'] = df['genres'].str.split(',').str[0]
df

In [ ]:
#Für eine schönere Darstellung entfernen wir noch die Sonderzeichen

import re

def clean_Sonderzeichen(values):
    cleaned_values = []
    for x in values:
        cleaned_value = re.sub(r'[^a-zA-Z]', '', x)
        if cleaned_value:
            cleaned_values.append(cleaned_value)
    return ''.join(cleaned_values)

# Wir wenden die Funktion für die Spalte genres und production_countries an

df['genres'] = df['genres'].apply(lambda x: clean_Sonderzeichen(x))

df['production_countries'] = df['production_countries'].apply(lambda x: clean_Sonderzeichen(x))
df


In [ ]:
#Wir haben mehr als zwei Ausprägungen, daher nutzen wir One Hot Encoding

df.genres.value_counts()

In [ ]:
#Wir haben zwei Spalten, die keinem Genre zugeordnet sind, daher füllen wir diese manuell auf

df[df['genres'] == '']

In [ ]:
"""Die beiden Filme werden als TV Show eingeordnet, da wir diese Ausprägung aber bisher nicht haben setzen wir beide
Filme auf die nächst passenede Ausprägung documentation"""

#Wir setzen das Genre für den ersten Film auf documentation

rows = df[df['name'] == 'David_Letterman']

for index, row in rows.iterrows():
    if row['genres'] == '':
        df.loc[index, 'genres'] = 'documentation'

#Das Genre für den zweiten Film setzen wir auch auf documentation

rows_2 = df[df['name'] == 'Carol_Burnett']

for index, row in rows_2.iterrows():
    if row['genres'] == '':
        df.loc[index, 'genres'] = 'documentation'

In [ ]:
#Unser Code war erfolgreich, die 2 Ausprägungen ohne Genre sind weg und bei documentation sind 2 Werte mehr
df.genres.value_counts()

In [ ]:
#Überprüfen, wie viele Genres aufgezählt sind
anzahl_eindeutige_werte = df['genres'].nunique()
anzahl_eindeutige_werte

In [ ]:
#Wir wenden nun One Hot Encoding an, löschen die genres Spalte und hängen die neuen an unseren Datensatz an

df = df.drop(columns='genres').join(pd.get_dummies(df.genres, drop_first = True))
df

In [ ]:
#'action' ist die Referenzkategorie
#Überprüfen der Werte in den Dummy-Spalten mit dem Count von vor Dummybildung
#drama            1026
#comedy            956
#documentation     377
#thriller          253
#scifi             190
#romance           176

summe1=df['drama'].sum()
summe2=df['comedy'].sum()
summe3=df['documentation'].sum()
summe4=df['thriller'].sum()
summe5=df['scifi'].sum()
summe6=df['romance'].sum()

print(summe1)
print(summe2)
print(summe3)
print(summe4)
print(summe5)
print(summe6)

###Dummy kodieren Production Countries

Leonie

In [ ]:
#Überprüfen, wie viele Länder aufgezählt sind
anzahl_eindeutige_werte = df['production_countries'].nunique()
anzahl_eindeutige_werte

In [ ]:
#Wir wiederholen die Schritte für Production Countries

df.production_countries.value_counts()

In [ ]:
#Wir haben sehr viele Kategorien, die nur einmal vorkommen um unser Modell nicht unnötig kompliziert zu machen fassen wir diese zusammen
#Damit es überschaubar bleibt setzen wir den Schwellenwert auf 100
#Die Länder kommen schließlich auch häufig vor, da die Filme mehrfach im Datensatz vorkommen, da jeder Schauspieler zu den Filmen genannt ist

threshold = 10

count = df.production_countries.value_counts()

infrequent_categories = count[count < threshold].index

df.loc[df['production_countries'].isin(infrequent_categories), 'production_countries'] = 'Other_countries'

In [ ]:
df.production_countries.value_counts()

In [ ]:
#Wir haben noch 70 Werte die keinem Land zugeordnet sind, diese setzten wir ebenfalls auf Other

df['production_countries'] = df['production_countries'].replace('', 'Other_countries')

In [ ]:
count=df.production_countries.value_counts()
count

In [ ]:
# wir speichern die Spaltennamen in einer Liste ab, um die Anzahl hinzufügenden Spalten zu überprüfen
p_countries_list = count.index.tolist()
p_countries_list
#Anzahl der Listenelemente ergibt die Anzahl der zu ergänzenden Spalten/Dummies:
anzahl_elemente = len(p_countries_list)
anzahl_elemente

#Ausgabe: 37 -> es müssen 36 neue Spalten hinzugefügt werden

In [ ]:
#Wir wenden nun One Hot Encoding an, löschen die production countries Spalte und hängen die neuen an unseren Datensatz an

df = df.drop(columns='production_countries').join(pd.get_dummies(df.production_countries, drop_first = True))
df.head(5)

In [ ]:
#Überprüfen der Werte in den Dummy-Spalten mit dem Count von vor Dummybildung
#US                 1302
#Other_countries     516
#IN                  414
#JP                  212
#KR                  171
#GB                  152
#ES                  126

summe1=df['US'].sum()
summe2=df['Other_countries'].sum()
summe3=df['IN'].sum()
summe4=df['JP'].sum()
summe5=df['KR'].sum()
summe6=df['GB'].sum()
summe7=df['ES'].sum()

print(summe1)
print(summe2)
print(summe3)
print(summe4)
print(summe5)
print(summe6)
print(summe7)

##Kodierung der Spalte Rolle

Leonie

In [ ]:
#Wir schauen zunächst wieder welche Werte die Spalte role hat

df.role.value_counts()

In [ ]:
#Wir haben nur Schauspieler im Datensatz, daher können wir diese Spalte einfach entfernen
df = df.drop(['role'], 1)
df

##Kodierung Schauspieler & Top 100 Schauspieler

Madeleine

Wir werden für die Top-100 Schauspieler binäre Spalten ergänzen.

In [ ]:
#Wir wollen einmal zählen, wie oft jeder Schauspieler im Datensatz vorkommt, um die Top-10 Schauspieler ausfindig zu machen

aufgeteilt = df['name'].str.split(', ')

# Zusammenführen der aufgeteilten Werte zu einer Liste
liste_werte = aufgeteilt.explode().tolist()

# Eindeutige Werte zählen
count = pd.Series(liste_werte).value_counts()

# Ausgabe der eindeutigen Werte und ihrer Häufigkeiten
count

In [ ]:
#hier lassen wir uns einmal die Verteilung anzeigen, wie die Anzahl der Schauspieler im Datensatz verteilt ist
#damit können wir definieren, wo wir unsere Grenze der Top-Schauspieler definieren möchten
frequency_count = count.value_counts().sort_index()
print(frequency_count)

In [ ]:
sum_total=frequency_count.sum().sum()
sum_total

In [244]:
#wir werden für unsere weiteren Analysen lediglich die Top-100 Schauspieler betrachten
top_100_names = count.head(100)
top_100_names

Shah_Rukh_Khan         23
Boman_Irani            22
Kareena_Kapoor_Khan    19
Aamir_Khan             18
Yuki_Kaji              17
                       ..
Ashleigh_Ball           9
Aoi_Yuki                9
Koji_Yusa               9
Tomokazu_Sugita         9
Graham_Chapman          9
Length: 100, dtype: int64

In [245]:
#Wir speichern die Top 100 Schauspieler in einer Liste ab

schauspieler_liste = top_100_names.index.tolist()
schauspieler_liste

['Shah_Rukh_Khan',
 'Boman_Irani',
 'Kareena_Kapoor_Khan',
 'Aamir_Khan',
 'Yuki_Kaji',
 'Priyanka_Chopra_Jonas',
 'Junichi_Suwabe',
 'Nawazuddin_Siddiqui',
 'Kenjiro_Tsuda',
 'Rani_Mukerji',
 'Amitabh_Bachchan',
 'Grey_DeLisle',
 'Anupam_Kher',
 'Maaya_Sakamoto',
 'Naseeruddin_Shah',
 'Fred_Tatasciore',
 'Takahiro_Sakurai',
 'Kari_Wahlgren',
 'John_Abraham',
 'Akshay_Kumar',
 'Donald_Trump',
 'Brijendra_Kala',
 'Murali_Sharma',
 'Om_Puri',
 'Takehito_Koyasu',
 'Tiffany_Haddish',
 'John_Paul_Tremblay',
 'Terry_Jones',
 'Fred_Armisen',
 'Robb_Wells',
 'Tabu',
 'Rajkummar_Rao',
 'Mike_Smith',
 'Mamoru_Miyano',
 'Paresh_Rawal',
 'Ajay_Devgn',
 'Kevin_Michael_Richardson',
 'Ranbir_Kapoor',
 'Kappei_Yamaguchi',
 'Nassar',
 'Pankaj_Tripathi',
 'John_Cleese',
 'Mariya_Ise',
 'Ron_Funches',
 'Anil_Kapoor',
 'Chris_Parnell',
 'Hiroaki_Hirata',
 'Johnny_Lever',
 'Rishi_Kapoor',
 'Adam_Sandler',
 'Kim_Mi-kyeong',
 'Tara_Strong',
 'Ken_Jeong',
 'Koichi_Yamadera',
 'Daisuke_Ono',
 'Sharat_Saxena',


In [246]:
#Anzahl der Listenelemente ergibt die Anzahl der zu ergänzenden Spalten:
anzahl_elemente = len(schauspieler_liste)

print(anzahl_elemente)

100


In [247]:
# Erzeugen eines DataFrame aus der Liste
neue_spalten_df = pd.DataFrame(columns=schauspieler_liste)
neue_spalten_df

# Verketten (Concatenation) des bestehenden DataFrames mit dem neuen DataFrame
df = pd.concat([df, neue_spalten_df], axis=1)
df

,name,character,description,runtime,imdb_score,imdb_votes,title,tmdb_score,animation,comedy,...,Vijay_Raaz,Sonam_Kapoor,Adil_Hussain,Nobunaga_Shimazaki,Manisha_Koirala,Ashleigh_Ball,Aoi_Yuki,Koji_Yusa,Tomokazu_Sugita,Graham_Chapman
0,"Yoo_Ah-in, Park_Shin-hye, Lee_Hyun-wook, Jin_S...","Oh Joon-woo, Kim Yoo-bin, Sang-chul, Elena Kim...","As a grisly virus rampages a city, a lone man ...",98.0,6.3,38313.0,#Alive,7.3,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Helen_Mirren, Anne_Frank, Martina_Gatti, Arian...","Narrator, Self, Self, Self",One single Anne Frank moves us more than the c...,92.0,6.5,1548.0,#AnneFrank. Parallel Stories,7.1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Adipati_Dolken, Vanesha_Prescilla, Rendi_Jhon,...","Ditto, Ayu, Darma, Mili, Dila, Rifnu, Aca, Iqb...","Pining for his high school crush for years, a ...",102.0,6.8,714.0,#FriendButMarried,6.7,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Adipati_Dolken, Mawar_Eva_De_Jongh, Vonny_Corn...","Ditto, Ayu, Ayu's Mother, Ditto's Mother, Ditt...",As Ayu and Ditto finally transition from best ...,104.0,6.9,311.0,#FriendButMarried 2,7.5,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Nesta_Cooper, Keith_Powers, Alicia_Sanz, Jake_...","Dani Barnes, Cameron Drake, Alexa Medina, Fred...",When nerdy high schooler Dani finally attracts...,99.0,5.1,6357.0,#realityhigh,6.4,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3945,"Koki_Uchiyama, Mikako_Komatsu, Rie_Takahashi, ...","Doujima Daisuke (voice), Milo (voice), Chang L...",An isolated city. An unknown enemy. An uncerta...,23.0,5.9,612.0,revisions,7.0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3946,"Andrew_Garfield, Alexandra_Shipp, Robin_de_Jes...","Jonathan Larson, Susan Wilson, Michael, Karess...","On the cusp of his 30th birthday, Jonathon Lar...",121.0,7.5,98370.0,"tick, tick... BOOM!",7.8,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3947,"Secun_de_la_Rosa, Estefanía_de_los_Santos, Mar...","Segundo, María, Rossi de la Vega, Carla, Pili,...","When her estranged mother suddenly dies, a wom...",81.0,3.9,404.0,"¡Ay, mi madre!",4.3,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3948,"Ada_Ameh, Beverly_Osu, Blossom_Chukwujekwu, Bu...","Titi, Peju, Emeka, Beauty, Tony, Sheriff, Simo...","In Lagos, Nigeria, young, naive Nigerian journ...",106.0,5.5,791.0,Òlòtūré,6.2,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [248]:
# Neue Schauspieler-Spalten abgleichen und entsprechend mit Werten befüllen, falls in Spalte 'name' enthalten:
for index, row in df.iterrows():
    names = row['name'].split(', ')  # Namen extrahieren

    for name in names:
        if name in schauspieler_liste:
            df.at[index, name] = 1  # Name in neu ergänzter Spalte gefunden
        else:
            df.at[index, 'Other_actors'] = 1  # Name in neuer Spalte "Other_actors" hinzufügen

df

,name,character,description,runtime,imdb_score,imdb_votes,title,tmdb_score,animation,comedy,...,Sonam_Kapoor,Adil_Hussain,Nobunaga_Shimazaki,Manisha_Koirala,Ashleigh_Ball,Aoi_Yuki,Koji_Yusa,Tomokazu_Sugita,Graham_Chapman,Other_actors
0,"Yoo_Ah-in, Park_Shin-hye, Lee_Hyun-wook, Jin_S...","Oh Joon-woo, Kim Yoo-bin, Sang-chul, Elena Kim...","As a grisly virus rampages a city, a lone man ...",98.0,6.3,38313.0,#Alive,7.3,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,"Helen_Mirren, Anne_Frank, Martina_Gatti, Arian...","Narrator, Self, Self, Self",One single Anne Frank moves us more than the c...,92.0,6.5,1548.0,#AnneFrank. Parallel Stories,7.1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,"Adipati_Dolken, Vanesha_Prescilla, Rendi_Jhon,...","Ditto, Ayu, Darma, Mili, Dila, Rifnu, Aca, Iqb...","Pining for his high school crush for years, a ...",102.0,6.8,714.0,#FriendButMarried,6.7,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,"Adipati_Dolken, Mawar_Eva_De_Jongh, Vonny_Corn...","Ditto, Ayu, Ayu's Mother, Ditto's Mother, Ditt...",As Ayu and Ditto finally transition from best ...,104.0,6.9,311.0,#FriendButMarried 2,7.5,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,"Nesta_Cooper, Keith_Powers, Alicia_Sanz, Jake_...","Dani Barnes, Cameron Drake, Alexa Medina, Fred...",When nerdy high schooler Dani finally attracts...,99.0,5.1,6357.0,#realityhigh,6.4,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3945,"Koki_Uchiyama, Mikako_Komatsu, Rie_Takahashi, ...","Doujima Daisuke (voice), Milo (voice), Chang L...",An isolated city. An unknown enemy. An uncerta...,23.0,5.9,612.0,revisions,7.0,1,0,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3946,"Andrew_Garfield, Alexandra_Shipp, Robin_de_Jes...","Jonathan Larson, Susan Wilson, Michael, Karess...","On the cusp of his 30th birthday, Jonathon Lar...",121.0,7.5,98370.0,"tick, tick... BOOM!",7.8,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3947,"Secun_de_la_Rosa, Estefanía_de_los_Santos, Mar...","Segundo, María, Rossi de la Vega, Carla, Pili,...","When her estranged mother suddenly dies, a wom...",81.0,3.9,404.0,"¡Ay, mi madre!",4.3,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3948,"Ada_Ameh, Beverly_Osu, Blossom_Chukwujekwu, Bu...","Titi, Peju, Emeka, Beauty, Tony, Sheriff, Simo...","In Lagos, Nigeria, young, naive Nigerian journ...",106.0,5.5,791.0,Òlòtūré,6.2,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [249]:
#Die NaN-Werte mit 0 auffüllen und Spalte 'name' löschen
df = df.fillna(0)
df = df.drop('name', axis=1)
df

,character,description,runtime,imdb_score,imdb_votes,title,tmdb_score,animation,comedy,crime,...,Sonam_Kapoor,Adil_Hussain,Nobunaga_Shimazaki,Manisha_Koirala,Ashleigh_Ball,Aoi_Yuki,Koji_Yusa,Tomokazu_Sugita,Graham_Chapman,Other_actors
0,"Oh Joon-woo, Kim Yoo-bin, Sang-chul, Elena Kim...","As a grisly virus rampages a city, a lone man ...",98.0,6.3,38313.0,#Alive,7.3,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
1,"Narrator, Self, Self, Self",One single Anne Frank moves us more than the c...,92.0,6.5,1548.0,#AnneFrank. Parallel Stories,7.1,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
2,"Ditto, Ayu, Darma, Mili, Dila, Rifnu, Aca, Iqb...","Pining for his high school crush for years, a ...",102.0,6.8,714.0,#FriendButMarried,6.7,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
3,"Ditto, Ayu, Ayu's Mother, Ditto's Mother, Ditt...",As Ayu and Ditto finally transition from best ...,104.0,6.9,311.0,#FriendButMarried 2,7.5,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
4,"Dani Barnes, Cameron Drake, Alexa Medina, Fred...",When nerdy high schooler Dani finally attracts...,99.0,5.1,6357.0,#realityhigh,6.4,0,1,0,...,0,0,0,0,0,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3945,"Doujima Daisuke (voice), Milo (voice), Chang L...",An isolated city. An unknown enemy. An uncerta...,23.0,5.9,612.0,revisions,7.0,1,0,0,...,0,0,1,0,0,0,0,0,0,1.0
3946,"Jonathan Larson, Susan Wilson, Michael, Karess...","On the cusp of his 30th birthday, Jonathon Lar...",121.0,7.5,98370.0,"tick, tick... BOOM!",7.8,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
3947,"Segundo, María, Rossi de la Vega, Carla, Pili,...","When her estranged mother suddenly dies, a wom...",81.0,3.9,404.0,"¡Ay, mi madre!",4.3,0,1,0,...,0,0,0,0,0,0,0,0,0,1.0
3948,"Titi, Peju, Emeka, Beauty, Tony, Sheriff, Simo...","In Lagos, Nigeria, young, naive Nigerian journ...",106.0,5.5,791.0,Òlòtūré,6.2,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0


In [250]:
#Umwandeln der Werte der Spalte 'Other_actors' von float zu integer:
df['Other_actors']=df['Other_actors'].astype(int)

In [251]:
#Überprüfen der Werte in den Dummy-Spalten mit dem Count von vor Dummybildung
#Shah_Rukh_Khan         23
#Boman_Irani            22
#Kareena_Kapoor_Khan    19
#Aamir_Khan             18
#Yuki_Kaji              17
#Aoi_Yuki                9
#Koji_Yusa               9
#Tomokazu_Sugita         9
#Graham_Chapman          9

summe1=df['Shah_Rukh_Khan'].sum()
summe2=df['Boman_Irani'].sum()
summe3=df['Kareena_Kapoor_Khan'].sum()
summe4=df['Aamir_Khan'].sum()
summe5=df['Yuki_Kaji'].sum()
summe6=df['Aoi_Yuki'].sum()
summe7=df['Koji_Yusa'].sum()
summe8=df['Tomokazu_Sugita'].sum()
summe9=df['Graham_Chapman'].sum()

print(summe1)
print(summe2)
print(summe3)
print(summe4)
print(summe5)
print(summe6)
print(summe7)
print(summe8)
print(summe9)

23
22
19
18
17
9
9
9
9


In [252]:
#Datei für Visualisierung abspeichern
df.to_csv("Daten_kodiert_fuer_Visualisierung.csv", index=False)

#Zielvariable berechnen
Leonie

In [253]:
'''Unsere Zielvariable für die Regression ist der imdb und tmdb Score, diese möchten wir in einem gewichteten Mittelwert
(75% imdb score 25% tmdb score) kodieren'''

weight_imdb = 0.75
weight_tmdb = 0.25

df['target_variable'] = ((df['imdb_score'] * weight_imdb) + (df['tmdb_score'] * weight_tmdb))

<ipython-input-253-f21276290bfa>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['target_variable'] = ((df['imdb_score'] * weight_imdb) + (df['tmdb_score'] * weight_tmdb))


In [254]:
#Die ursprünglichen Spalten löschen wir

df = df.drop(columns=['imdb_score', 'tmdb_score'])

In [255]:
df.isna().any()

character          False
description        False
runtime            False
imdb_votes         False
title              False
                   ...  
Koji_Yusa          False
Tomokazu_Sugita    False
Graham_Chapman     False
Other_actors       False
target_variable    False
Length: 160, dtype: bool

In [256]:
df

,character,description,runtime,imdb_votes,title,animation,comedy,crime,documentation,drama,...,Adil_Hussain,Nobunaga_Shimazaki,Manisha_Koirala,Ashleigh_Ball,Aoi_Yuki,Koji_Yusa,Tomokazu_Sugita,Graham_Chapman,Other_actors,target_variable
0,"Oh Joon-woo, Kim Yoo-bin, Sang-chul, Elena Kim...","As a grisly virus rampages a city, a lone man ...",98.0,38313.0,#Alive,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,6.550
1,"Narrator, Self, Self, Self",One single Anne Frank moves us more than the c...,92.0,1548.0,#AnneFrank. Parallel Stories,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,6.650
2,"Ditto, Ayu, Darma, Mili, Dila, Rifnu, Aca, Iqb...","Pining for his high school crush for years, a ...",102.0,714.0,#FriendButMarried,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,6.775
3,"Ditto, Ayu, Ayu's Mother, Ditto's Mother, Ditt...",As Ayu and Ditto finally transition from best ...,104.0,311.0,#FriendButMarried 2,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,7.050
4,"Dani Barnes, Cameron Drake, Alexa Medina, Fred...",When nerdy high schooler Dani finally attracts...,99.0,6357.0,#realityhigh,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,5.425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3945,"Doujima Daisuke (voice), Milo (voice), Chang L...",An isolated city. An unknown enemy. An uncerta...,23.0,612.0,revisions,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,6.175
3946,"Jonathan Larson, Susan Wilson, Michael, Karess...","On the cusp of his 30th birthday, Jonathon Lar...",121.0,98370.0,"tick, tick... BOOM!",0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,7.575
3947,"Segundo, María, Rossi de la Vega, Carla, Pili,...","When her estranged mother suddenly dies, a wom...",81.0,404.0,"¡Ay, mi madre!",0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,4.000
3948,"Titi, Peju, Emeka, Beauty, Tony, Sheriff, Simo...","In Lagos, Nigeria, young, naive Nigerian journ...",106.0,791.0,Òlòtūré,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,5.675


#Speichern der Daten

In [257]:
df.to_csv("Daten_kodiert_fuer_modelle_dummycodierung_V5.csv", index=False)